In [1]:
# pip install tflearn

In [2]:
# Created the neural network module which will be used to detect Hindi and Non-Hindi characters from given images.

# The attached output file will show the classified images, which will show the classification as below:
# 1 = Non-Hindi Character Images
# 0 = Hindi Character Images

# form the testing data samples, for example: 
# "85.png : ": "1", means 85.png does not have Hindi character.
# "15.png : ": "0", means 15.png has Hindi character.


In [3]:
# Import Required Libraries

import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel Bühler for this suggestion


In [4]:
TRAIN_DIR = 'data/training/training/hi'
TRAIN_DIR2 = 'data/training/training/background'
TEST_DIR = 'data/test/test'
IMG_SIZE = 50
LR = 1e-3   
# Keeping learning rate as minimun as possible so that it will update weights and we could get global optimal solution.
# Learning rate would help us to converge steadily with very small jump.

MODEL_NAME = 'pradnya-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which.

In [5]:
# Read all training images and convert it into numPy array after resize.
def create_train_data():
    training_data = []

    for img in tqdm(os.listdir(TRAIN_DIR)):
        
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array([1,0])])
        
    for img in tqdm(os.listdir(TRAIN_DIR2)):
        
        path = os.path.join(TRAIN_DIR2,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array([0,1])])
        
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [6]:
train_data = create_train_data()

100%|████████████████████████████████████████████████████████████████████████████| 4450/4450 [00:02<00:00, 1960.37it/s]
C:\Users\Pradnya\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [7]:
# Read all testing images and convert it into numPy array after resize.
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [8]:
process_test_data()

100%|████████████████████████████████████████████████████████████████████████████████| 98/98 [00:00<00:00, 4082.45it/s]


[[array([[ 31,  27,  25, ...,  62,  39,  35],
         [ 35,  32,  30, ...,  87,  62,  75],
         [ 31,  29,  28, ...,  94,  76, 118],
         ...,
         [162, 152, 147, ..., 175, 169, 159],
         [152, 154, 159, ..., 175, 173, 182],
         [157, 155, 164, ..., 172, 162, 185]], dtype=uint8), '31'],
 [array([[50, 54, 53, ..., 29, 30, 32],
         [49, 51, 53, ..., 29, 28, 27],
         [52, 51, 51, ..., 37, 33, 25],
         ...,
         [59, 68, 68, ..., 27, 27, 27],
         [61, 61, 63, ..., 27, 26, 25],
         [60, 51, 55, ..., 28, 26, 23]], dtype=uint8), '42'],
 [array([[118, 121, 120, ..., 180, 157, 150],
         [123, 124, 120, ..., 153, 147, 162],
         [121, 123, 122, ..., 145, 152, 163],
         ...,
         [ 92, 111, 142, ..., 113, 105, 122],
         [111, 162, 178, ..., 118, 117, 148],
         [132, 151, 177, ..., 129, 136, 163]], dtype=uint8), '48'],
 [array([[131, 136, 157, ...,  69,  67,  40],
         [148, 158, 154, ...,  99,  93,  59],
        

In [9]:
train = train_data[:-875]
test = train_data[-875:]

In [10]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [11]:
# Import TenserFlow for Deep Learning Module. 
# TensorFlow is a free and open-source software library for machine learning and artificial intelligence. 
# It can be used across a range of tasks but has a particular focus on training and inference of deep neural networks.

import tensorflow as tf

In [12]:
# TFlearn is a modular and transparent deep learning library built on top of Tensorflow.

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [13]:
# Lets create the module as per our requirement.
# We have tried different combinations of neural network layers and come to on conclusion that the below
# neural network works with higher accuracy.

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')   
# Softmax will convert our output into 2 classes i.e (Hindi and Non Hindi).
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[:-875]
test = train_data[-875:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=4, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 1263  | total loss: 0.04289 | time: 9.060s
| Adam | epoch: 004 | loss: 0.04289 - acc: 0.9956 -- iter: 4992/5000
Training Step: 1264  | total loss: 0.03866 | time: 10.185s
| Adam | epoch: 004 | loss: 0.03866 - acc: 0.9960 | val_loss: 0.02763 - val_acc: 0.9909 -- iter: 5000/5000
--


In [14]:
# from above neural network we have achieved an accuracy around 99.5%. As per accuracy this is very good module.
# When we test this module on Testing images, it will definetly give us accurate results. 
# Lets save the module and try it on testing images.

In [15]:
model.save(MODEL_NAME)

INFO:tensorflow:C:\Users\Pradnya\competiton hackathon\pradnya-0.001-2conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
import random
import json
# from utils.io import write_json
json_dict = {}
outputfile = 'sample_output_file.json'


# 1 = Non-Hindi
# 0 = Hindi
def generate_output(filename, result):
    filename = str(filename) +".png : "
    #print(filename)
    json_dict[filename] = str(result)
    #print(json_dict)
    
def write_output():
    with open(outputfile, 'w') as outfile:
        json.dump(json_dict,outfile)


In [22]:
import matplotlib.pyplot as plt

# Create testing data in numPy format
test_data = process_test_data()

fig=plt.figure(figsize=(150, 200))

for num,data in enumerate(test_data):
    #print(num)
    filename = data[1]
    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(15,7,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    
    # Provide an Image to deep learning module and predict the class. 
    model_out = model.predict([data])[0]
    # Output class value for each testing image
    


    if np.argmax(model_out) == 1: 
        str_label='Non-Hindi'
        generate_output(filename, 1)
    else: 
        str_label='Hindi'
        generate_output(filename, 0)
        
    #plt.imshow(orig,cmap='gray')
    #plt.axis('on')
    #plt.title(str_label)
    #plt.show()
    y.imshow(orig,cmap='gray')
    plt.title(str_label, fontsize=100, color="black")
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()
write_output()

100%|████████████████████████████████████████████████████████████████████████████████| 98/98 [00:00<00:00, 3063.86it/s]
